In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, SpatialDropout1D, GRU
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer

from keras.preprocessing import sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline


Using TensorFlow backend.


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
## Loading Data - labelled data:

data_1 = pd.ExcelFile('RNN_Tranzact_Rohan_1_Regrouped_with_supplier.xlsx')
tranzact_data_1 = data_1.parse(0)

tranzact_data_1.tail()

,supplier_item_id,to_company_name,supplier_item_id.1,supplier_item_id.2,product,supplier,Done,Atul Sugg.,sub_type,material,process,grade,spec,brand
13552,86390,CESARE BONETTI INTERNATIONAL PVT. LTD.,86390,"YOKE SLEEVE;8"" # 150;GTV; B150 C62300",YOKE SLEEVE,CESARE BONETTI INDIA PVT. LTD.,278.0,NaN,NaN,NaN,NaN,C62300,NaN,NaN
13553,89723,CESARE BONETTI INTERNATIONAL PVT. LTD.,89723,"YOKE SLEEVE 1½"" #2700 BLY;B150 C62300",YOKE SLEEVE,CESARE BONETTI INDIA PVT. LTD.,278.0,NaN,NaN,NaN,NaN,C62300,NaN,NaN
13554,89995,CESARE BONETTI INTERNATIONAL PVT. LTD.,89995,"YOKE SLEEVE;4"" #150 GTV; B150 C62300",YOKE SLEEVE,CESARE BONETTI INDIA PVT. LTD.,278.0,NaN,NaN,NaN,NaN,C62300,NaN,NaN
13555,117819,WAAREE INDUSTRIES PVT. LTD.,117819,"YOKE SLEEVE;8"" # 150;GTV; A439 D2",cicasting,Waaree Industries Pvt.Ltd.,278.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13556,169292,CESARE BONETTI INTERNATIONAL PVT. LTD.,169292,"YOKE SLEEVE; 1"";#1500;CBD;A582 T416",YOKE SLEEVE,CESARE BONETTI INDIA PVT. LTD.,278.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
## Loading Data - un-labelled data:

data_2 = pd.ExcelFile('14Mar_Gorky.xlsx')
tranzact_data_new_1 = data_2.parse(0)

tranzact_data_new_1.tail()

,buyer_item_id,buyer_itemid,buyer_item_name,from_company_id,from_company_name,to_company_id,to_company_name
17763,289052,PLYBAG_10X15,POLYBAG_10X15 INCH_PP,13872,R-Tech Products & Packaging Pvt Ltd,16730,GLORY PACK INDUSTRIES
17764,289639,HANDLENUTM12X5MMXX,"HANDLE NUT 1"" #600,HTB,A194 Gr8",842,CESARE BONETTI INDIA PVT. LTD.,7434,Shah Brothers
17765,48512,RDBR50 (SS431),ROUND BAR OD50 ASTM A276 TYPE 431,114,Entech Controls,921,ALPESH METALS
17766,57696,PGX618101601202200,"PTV 6"" #300 RF BVE WCB HW IBR",7506,CESARE BONETTI INTERNATIONAL PVT. LTD.,842,CESARE BONETTI INDIA PVT. LTD.
17767,274954,PPBOX_ST5,PP_MONOBOX_ST5,13872,R-Tech Products & Packaging Pvt Ltd,15527,Shree Arun Packaging Co. Pvt. Ltd.


In [5]:
## Picking up required info from labelled data:

col = ['to_company_name', 'supplier_item_id.2', 'product', 'supplier']
tranzact_data_2 = tranzact_data_1[col]
tranzact_data_2.columns = ['OEM', 'item', 'product', 'supplier']

## Picking up required info from un-labelled data:

col = ['from_company_name', 'buyer_item_name', 'to_company_name']
tranzact_data_new_2 = tranzact_data_new_1[col]
tranzact_data_new_2.columns = ['OEM', 'item', 'supplier']

## Merging two data frames :

tranzact_data_combined = pd.concat([tranzact_data_2, tranzact_data_new_2], axis=0, ignore_index=True)



## Removing duplicate (item + supplier) combinations :

tranzact_data_combined['find_duplicate'] = tranzact_data_combined['item'].map(str) + \
                                            tranzact_data_combined['supplier'].map(str)

tranzact_data_combined.drop_duplicates(subset='find_duplicate', keep = 'first', inplace = True)
tranzact_data_combined = tranzact_data_combined.reset_index(drop=True)

# Creating separate dataframe for Cesare Bonetti International Pvt. Ltd. since these are unlabelled :

#tranzact_data_2_cesare_international = tranzact_data_2.loc[tranzact_data_2['OEM'] \
#                                                               == 'CESARE BONETTI INTERNATIONAL PVT. LTD.']

#tranzact_data_labelled = tranzact_data_2.drop(tranzact_data_2_cesare_international.index)

#tranzact_data_2_cesare_international = tranzact_data_2_cesare_international.reset_index(drop=True)
#tranzact_data_labelled = tranzact_data_labelled.reset_index(drop=True)

#print(len(tranzact_data_2_cesare_international))
        
print(len(tranzact_data_combined))

tranzact_data_combined.tail()

30803


,OEM,item,product,supplier,find_duplicate
30798,R-Tech Products & Packaging Pvt Ltd,POLYBAG_10X15 INCH_PP,NaN,GLORY PACK INDUSTRIES,POLYBAG_10X15 INCH_PPGLORY PACK INDUSTRIES
30799,CESARE BONETTI INDIA PVT. LTD.,"HANDLE NUT 1"" #600,HTB,A194 Gr8",NaN,Shah Brothers,"HANDLE NUT 1"" #600,HTB,A194 Gr8Shah Brothers"
30800,Entech Controls,ROUND BAR OD50 ASTM A276 TYPE 431,NaN,ALPESH METALS,ROUND BAR OD50 ASTM A276 TYPE 431ALPESH METALS
30801,CESARE BONETTI INTERNATIONAL PVT. LTD.,"PTV 6"" #300 RF BVE WCB HW IBR",NaN,CESARE BONETTI INDIA PVT. LTD.,"PTV 6"" #300 RF BVE WCB HW IBRCESARE BONETTI IN..."
30802,R-Tech Products & Packaging Pvt Ltd,PP_MONOBOX_ST5,NaN,Shree Arun Packaging Co. Pvt. Ltd.,PP_MONOBOX_ST5Shree Arun Packaging Co. Pvt. Ltd.


In [30]:
## make all in lower case :

tranzact_data_combined['OEM'] = tranzact_data_combined['OEM'].map(lambda x: x if type(x)!=str else x.lower()) 
tranzact_data_combined['item'] = tranzact_data_combined['item'].map(lambda x: x if type(x)!=str else x.lower()) 
tranzact_data_combined['product'] = tranzact_data_combined['product'].map(lambda x: x if type(x)!=str else x.lower())

## Removing some punctuations:

punctuation = [':', ';', '!', '.', '_', ',', '+'] 

for i in punctuation:
    
    tranzact_data_combined['item']= tranzact_data_combined['item'].str.replace(i," ")

## 

## remove rows with any cell value = none

# tranzact_data_labelled_final = tranzact_data_labelled.dropna()
# tranzact_data_un_labelled_final = tranzact_data_un_labelled.dropna()

# tranzact_data_labelled_final = tranzact_data_labelled_final.reset_index(drop=True)
# tranzact_data_un_labelled_final = tranzact_data_un_labelled_final.reset_index(drop=True)

print(len(tranzact_data_combined))
tranzact_data_combined.tail()

30803


,OEM,item,product,supplier,find_duplicate
30798,r-tech products & packaging pvt ltd,polybag 10x15 inch pp,NaN,GLORY PACK INDUSTRIES,POLYBAG_10X15 INCH_PPGLORY PACK INDUSTRIES
30799,cesare bonetti india pvt. ltd.,"handle nut 1"" #600 htb a194 gr8",NaN,Shah Brothers,"HANDLE NUT 1"" #600,HTB,A194 Gr8Shah Brothers"
30800,entech controls,round bar od50 astm a276 type 431,NaN,ALPESH METALS,ROUND BAR OD50 ASTM A276 TYPE 431ALPESH METALS
30801,cesare bonetti international pvt. ltd.,"ptv 6"" #300 rf bve wcb hw ibr",NaN,CESARE BONETTI INDIA PVT. LTD.,"PTV 6"" #300 RF BVE WCB HW IBRCESARE BONETTI IN..."
30802,r-tech products & packaging pvt ltd,pp monobox st5,NaN,Shree Arun Packaging Co. Pvt. Ltd.,PP_MONOBOX_ST5Shree Arun Packaging Co. Pvt. Ltd.


In [31]:
from gensim.models import Word2Vec
from nltk import word_tokenize
from tqdm import tqdm

temp_corpus = tranzact_data_combined['item'].map(lambda x: x.split('.'))

corpus = []
for i in tqdm(range(len(temp_corpus))):
    for line in temp_corpus[i]:
        words = [x for x in line.split()]
        corpus.append(words)

100%|██████████| 30803/30803 [00:00<00:00, 46936.75it/s]


In [11]:
print(type(corpus))
print(corpus)

<class 'list'>
[['new', 'cylinder', '404', 'gas'], ['accumulator', '-', 'model', 'a-as', '5126', '(3/4"', 'connection)', '-', 'emerson', 'make'], ['accumulator', 'flokool', '3/4"', '400', 'psi', 'fksa596'], ['accumulator', '1', '5/8', 'connection'], ['accumulator', '7/8‘’'], ['accumulator', '13/8', "''"], ['accumulator', 'flokool', '5/8"', '400', 'psi', 'fksa596'], ['accumulator', '1-1/8'], ['accumulator', '2-1/8'], ['accumulator', '5216-7s'], ['buthyl', 'acetate'], ['ethyl', 'acetate'], ['npac(n-proyl', 'acetate)'], ['pma(p', 'm', 'acetate)'], ['flashback', 'arrester', 'acetylene', 'cylinder', 'mounted'], ['acetylene', 'cylinder', '4kg'], ['acetylene', 'gas'], ['meta-phosphoric', 'acid'], ['standard', 'ascorbic', 'acid'], ['protobond', 'acid', 'for', 'cleaning'], ['distilled', 'rice', 'bran', 'fatty', 'acid'], ['citric', 'acid'], ['uric', 'acid', 'des', 'kit', 'erba'], ['lauric', 'acid'], ['palm', 'fatty', 'acid'], ['consumables', '-', 'hil', 'acid', '5ltf'], ['spare', 'kit', 'iq', 'r

In [9]:
num_of_items = len(corpus)
num_of_words = 0
for line in corpus:
    num_of_words += len(line)

print('Num of sentences - %s'%(num_of_items))
print('Num of words - %s'%(num_of_words))

Num of sentences - 30803
Num of words - 217983


In [10]:
## items to word2vec :

emb_dim = 100

model_1 = Word2Vec(corpus, size = emb_dim, window=3, \
               min_count=1, negative = 15, iter = 10, workers = 10, sg=1)



In [12]:
print(model_1)

Word2Vec(vocab=20057, size=100, alpha=0.025)


In [24]:
## Splitting the combined data into labelled data and unlabelled data :

# tranzact_data_unlabelled = tranzact_data_combined.loc[tranzact_data_combined['product'] == Null]

tranzact_data_labelled = tranzact_data_combined.loc[pd.notnull(tranzact_data_combined['product'])]

tranzact_data_un_labelled = tranzact_data_combined.drop(tranzact_data_labelled.index)

tranzact_data_labelled = tranzact_data_labelled.reset_index(drop=True)
tranzact_data_un_labelled = tranzact_data_un_labelled.reset_index(drop=True)

print(len(tranzact_data_labelled))
print(len(tranzact_data_un_labelled))
tranzact_data_un_labelled.tail()

13556
17247


,OEM,item,product,supplier,find_duplicate
17242,r-tech products & packaging pvt ltd,polybag 10x15 inch pp,NaN,GLORY PACK INDUSTRIES,POLYBAG_10X15 INCH_PPGLORY PACK INDUSTRIES
17243,cesare bonetti india pvt. ltd.,"handle nut 1"" #600 htb a194 gr8",NaN,Shah Brothers,"HANDLE NUT 1"" #600,HTB,A194 Gr8Shah Brothers"
17244,entech controls,round bar od50 astm a276 type 431,NaN,ALPESH METALS,ROUND BAR OD50 ASTM A276 TYPE 431ALPESH METALS
17245,cesare bonetti international pvt. ltd.,"ptv 6"" #300 rf bve wcb hw ibr",NaN,CESARE BONETTI INDIA PVT. LTD.,"PTV 6"" #300 RF BVE WCB HW IBRCESARE BONETTI IN..."
17246,r-tech products & packaging pvt ltd,pp monobox st5,NaN,Shree Arun Packaging Co. Pvt. Ltd.,PP_MONOBOX_ST5Shree Arun Packaging Co. Pvt. Ltd.


In [25]:
## digitising 'product' and creating a dictionary for labelled data :

tranzact_data_labelled['product_id'] = tranzact_data_labelled['product'].factorize()[0]

product_id_df = tranzact_data_labelled[['product', 'product_id']].drop_duplicates().sort_values('product_id')
product_to_id = dict(product_id_df.values)
id_to_product = dict(product_id_df[['product_id', 'product']].values)

## removing puncuations from items of both labelled and un-labelled data :

# pd.options.mode.chained_assignment = None

# tranzact_data_labelled_final['item'] = tranzact_data_labelled_final['item'].str.replace(r'\W',' ')
# tranzact_data_un_labelled_final['item'] = tranzact_data_un_labelled_final['item'].str.replace(r'\W',' ')

## remove gaps between words to single gap

# tranzact_data_labelled_final['item'] = tranzact_data_labelled_final['item'].str.replace(r'\s+',' ')
# tranzact_data_un_labelled_final['item'] = tranzact_data_un_labelled_final['item'].str.replace(r'\s+',' ')

print(len(tranzact_data_labelled))
print(len(tranzact_data_un_labelled))
tranzact_data_labelled.tail()

13556
17247


,OEM,item,product,supplier,find_duplicate,product_id
13551,cesare bonetti international pvt. ltd.,"yoke sleeve 8"" # 150 gtv b150 c62300",yoke sleeve,CESARE BONETTI INDIA PVT. LTD.,"YOKE SLEEVE;8"" # 150;GTV; B150 C62300CESARE B...",156
13552,cesare bonetti international pvt. ltd.,"yoke sleeve 1½"" #2700 bly b150 c62300",yoke sleeve,CESARE BONETTI INDIA PVT. LTD.,"YOKE SLEEVE 1½"" #2700 BLY;B150 C62300CESARE BO...",156
13553,cesare bonetti international pvt. ltd.,"yoke sleeve 4"" #150 gtv b150 c62300",yoke sleeve,CESARE BONETTI INDIA PVT. LTD.,"YOKE SLEEVE;4"" #150 GTV; B150 C62300CESARE BON...",156
13554,waaree industries pvt. ltd.,"yoke sleeve 8"" # 150 gtv a439 d2",cicasting,Waaree Industries Pvt.Ltd.,"YOKE SLEEVE;8"" # 150;GTV; A439 D2Waaree Indus...",20
13555,cesare bonetti international pvt. ltd.,"yoke sleeve 1"" #1500 cbd a582 t416",yoke sleeve,CESARE BONETTI INDIA PVT. LTD.,"YOKE SLEEVE; 1"";#1500;CBD;A582 T416CESARE BONE...",156


In [29]:
tokenizer_obj_comb = Tokenizer()
tokenizer_obj_lab = Tokenizer()
tokenizer_obj_un_lab = Tokenizer()

tokenizer_obj_comb.fit_on_texts(tranzact_data_combined['item'])
tokenizer_obj_lab.fit_on_texts(tranzact_data_labelled['item'])
tokenizer_obj_un_lab.fit_on_texts(tranzact_data_un_labelled['item'])

## pad sequences :

max_length_comb = max([len(s.split()) for s in tranzact_data_combined['item']])
max_length_lab = max([len(s.split()) for s in tranzact_data_labelled['item']])
max_length_un_lab = max([len(s.split()) for s in tranzact_data_un_labelled['item']])

# min_length = min([len(s.split()) for s in total_item_labelled_unlabelled['item']])

## define vocabulary size :

vocab_size_comb = len(tokenizer_obj_comb.word_index) + 1
vocab_size_lab = len(tokenizer_obj_lab.word_index) + 1
vocab_size_un_lab = len(tokenizer_obj_un_lab.word_index) + 1

# total_item_tokens = tokenizer_obj.texts_to_sequences(total_item_labelled_unlabelled['item'])
# total_item_tokens_1 = tokenizer_obj_1.texts_to_sequences(tranzact_data_labelled_final['item'])

# total_item_pad = pad_sequences(total_item_tokens, maxlen = max_length, padding = 'post')
# total_item_pad_1 = pad_sequences(total_item_tokens_1, maxlen = max_length, padding = 'post')

print(max_length_comb)
print(max_length_lab)
print(max_length_un_lab)
print('==============')
print(vocab_size_comb)
print(vocab_size_lab)
print(vocab_size_un_lab)

104
50
104
15318
8679
11187


In [38]:
# results = set()
from collections import Counter

results = Counter()
tranzact_data_labelled['item'].str.lower().str.split().apply(results.update)
print(len(results))

print(results)


11017
Counter({'x': 2591, '-': 2340, 'mm': 2050, 'bar': 1156, 'wire': 948, '5': 865, 'steel': 863, 'spring': 838, 'gr': 797, '2': 710, 'ss': 690, 'cast': 680, '1': 610, '8': 586, 'wcb': 568, 'dia': 542, '3': 536, 'a193': 533, 'mesh': 508, 'nut': 499, 'gtv': 467, '00': 450, 'bolt': 426, 's': 417, 'pipe': 417, '304': 415, 'ring': 414, 'ibr': 414, 'screw': 400, 'body': 395, 'a182': 392, 'chromium': 390, 'stud': 389, 'for': 360, '4': 359, 'a479': 358, 'hex': 350, 'a216': 347, 'sp': 343, '0': 339, 'b7': 334, 'size': 334, 'plate': 334, 'casting': 332, 'ms': 324, 'valve': 322, 'st': 320, 'rod': 315, '6': 314, 'id': 308, 'ss304': 295, 'annealed': 288, 'thk': 278, 'washer': 277, 'type': 277, 'gasket': 276, 'ft': 261, 'pin': 253, 'hydraulic': 243, 'gland': 241, 'od': 238, 'rm': 237, 'part': 237, 'swg': 235, 'flange': 231, 'packing': 230, '316': 222, 'a194': 221, 'no': 216, 'a105n': 215, '#300': 214, 'coil': 212, '#150': 210, '10': 206, '20': 202, 'sheet': 202, '2"': 199, '50': 196, '1/2"': 196, 

In [40]:
wd_index = tokenizer_obj_lab.word_index
print(len(wd_index))
print(wd_index)


8678
{'i': 95, 'vehicle': 4524, 'entries': 4106, 'a563': 5585, 'speed': 1371, '245mm': 6762, 'grill': 1524, '803': 2155, '612': 3009, '3ply': 5829, 'm4p': 1722, '187k': 7424, 'seg': 3297, 'vrrm': 3328, '901': 2630, 'm2': 1401, 'spg': 287, 'x20': 2629, 'bt33': 4155, 'a439': 1609, 'setting': 7258, 'amplifier': 3696, 'packaging': 1190, '82': 2795, '0603': 292, '2600': 8224, '146': 3211, 'retaining': 2867, 'china': 3196, 'cv1500': 1568, 'with340x180x160': 6597, 'marked': 4110, '5x4mm': 7702, 'm24': 758, '003': 3131, 'sicop': 5374, 'butt': 1359, '1500uf': 4800, 'esd': 2583, '9t': 5736, 'd270': 2404, 'threaded': 673, '4”dial': 1222, '6215': 4580, 'flg': 4317, '1100033': 7987, 'ftc': 8282, '3070': 7322, 'deep': 1108, '24': 138, 'recovery': 5182, 'group': 8098, 'sm4': 3754, '195xthk': 5935, '718': 3048, 'txmr': 8191, '9700': 6183, '182': 3024, 'ts': 1238, 'rating': 2955, '6pin': 4900, 'converters': 3686, 'km55dr35': 7773, '12tx1250wx6000l': 7216, 'flap': 1297, 'wh': 1141, '120x90x15t': 6654, '

In [63]:
word_index = tokenizer_obj_1.word_index
print('Found %s unique tokens' % len(word_index))

Found 7948 unique tokens


In [67]:
print(word_index)

{'prmium': 5548, 'l310': 7300, '7090': 6245, 'a00000274': 3034, '2co': 3123, 'final': 2506, 'uric': 3879, '15k': 2637, '870': 5404, 'corrugated': 648, 'cesare': 7407, '3x1': 1766, '6': 17, 'horizontal': 1143, 'lead': 2041, 'apkt': 5903, 'gtq': 5708, 'allen': 319, 'oen': 7342, 'fevicol': 3934, 'wz': 6256, 'm42': 948, 'loggers': 7621, 'carb': 1670, '44mm': 5538, '3102r': 2982, '2z': 2845, 'microsystems': 3377, 'od80': 3992, 'rwh': 4923, 'is210': 183, 'stth212s': 3054, '3a': 895, '40x100x40mm': 4487, 'red': 229, 'processors': 5749, 'gland': 98, 'ele': 7439, 'mppt': 2234, '207': 6523, 'cr22': 2972, '1094': 5562, 'l25': 1177, 's': 19, 'pfte': 5556, 'p08f1803v': 6892, 'x750': 1609, '2330': 6502, 'compo': 886, 'polycarbonate': 2570, 'milk': 6759, '30deg': 4868, 'm3x6': 2022, 'iot': 6426, 'register': 5791, 'machines': 3098, 'cylindrical': 2840, '106mm': 5525, 'proyl': 3869, 'antistatic': 3693, '400mm': 2396, 'well': 1912, '60x30': 5259, '450x150x40': 3579, '18x11x60': 5974, 'donit': 2476, 'a24

In [61]:
wrd = 'prmium'
print(model_1.wv.word_vec(wrd))


[ 0.24179202  0.06549174 -0.38604274  0.2220079  -0.17099561 -0.00210161
 -0.19163914  0.02628648  0.1819302  -0.13675494  0.03219317 -0.00996133
 -0.05369972  0.2527915   0.2741835   0.12086015 -0.04894331 -0.16053675
 -0.00583438  0.00325824  0.06969016 -0.11515658  0.0805554   0.09401441
  0.2272013   0.03878359  0.27700585 -0.00347151  0.03068962  0.12152074
  0.20699036  0.09057083  0.06172016  0.08677412  0.16805473  0.0134774
 -0.09955977 -0.23010123  0.11060134  0.06406747  0.2174236  -0.03278314
  0.10400584  0.06704241 -0.21097016 -0.12631032 -0.05094669 -0.09503785
 -0.10903387 -0.0955196   0.22453734 -0.03931703 -0.17506832 -0.19794413
  0.3891973   0.35113436 -0.5200029   0.1512953  -0.04274882 -0.15493187
  0.07763566 -0.08534835  0.02209762  0.04802343 -0.04624603  0.12014635
 -0.162002   -0.21071538 -0.04798426  0.07724632 -0.19445488 -0.2439696
 -0.23706     0.02334258 -0.12904176 -0.14800826  0.02665363  0.13792053
  0.11965785  0.28002065]


In [78]:
j = 0

In [79]:
## prepare embeddings

## Preparing embedding matrix

nb_words = min(vocab_size, vocab_size_1)-1

embedding_matrix = np.zeros((nb_words, emb_dim))

    
for word, i in word_index.items():
        
        
    if word in model_1.wv.vocab:
            
        embedding_matrix[j] = model_1.wv.word_vec(word)
        j = j+1
    
    else:
                
        print(word)
        
        
print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))


iot
le18b22rd
uc18h7tft
18g16sk
ver1
txmr
1ohm
m66
935uh
le18b28pid
1755uh
le15k17cl
52uh
05w
54w
gaurds
le18h23ap
16uh
le18j27dg2
jsw
guards
le17h8ap
le18k1m
18b3sk
le17l2ap
le18j19ap
jsav150
le18h16dg2
le18l12dg2
89x74
atomfan4
48x38
le17f5wc
elettro
tft
Null word embeddings: 35


In [76]:
print(type(embedding_matrix))
print(embedding_matrix.shape)
print(embedding_matrix[0])

<class 'numpy.ndarray'>
(7948, 80)
[ 0.24179202  0.06549174 -0.38604274  0.2220079  -0.17099561 -0.00210161
 -0.19163914  0.02628648  0.1819302  -0.13675494  0.03219317 -0.00996133
 -0.05369972  0.25279149  0.27418351  0.12086015 -0.04894331 -0.16053675
 -0.00583438  0.00325824  0.06969016 -0.11515658  0.0805554   0.09401441
  0.2272013   0.03878359  0.27700585 -0.00347151  0.03068962  0.12152074
  0.20699036  0.09057083  0.06172016  0.08677412  0.16805473  0.0134774
 -0.09955977 -0.23010123  0.11060134  0.06406747  0.2174236  -0.03278314
  0.10400584  0.06704241 -0.21097016 -0.12631032 -0.05094669 -0.09503785
 -0.10903387 -0.0955196   0.22453734 -0.03931703 -0.17506832 -0.19794413
  0.38919729  0.35113436 -0.5200029   0.1512953  -0.04274882 -0.15493187
  0.07763566 -0.08534835  0.02209762  0.04802343 -0.04624603  0.12014635
 -0.162002   -0.21071538 -0.04798426  0.07724632 -0.19445488 -0.2439696
 -0.23706     0.02334258 -0.12904176 -0.14800826  0.02665363  0.13792053
  0.11965785  0.28

In [ ]:
## Total no. of unique words = vocab_size = 14707
## each word has got 150 parameters as per Embedding Dimension defined.
## Hence total Param # = 14707 * 150 = 2206050


In [35]:
from keras.models import Sequential

model_2 = Sequential()
model_2.add(Embedding(vocab_size_1, emb_dim, weights=[embedding_matrix],\
                    input_length=max_length, trainable=False))
model_2.add(SpatialDropout1D(0.2))
#model.add(LSTM(32, dropout=0.2, recurrent_dropout=0.2))
model_2.add(GRU(units=32, dropout=0.2, recurrent_dropout=0.2))
model_2.add(Dense(256, activation='relu'))
model_2.add(Dense(71, activation='softmax'))
model_2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

model_2.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 121, 100)          794900    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 121, 100)          0         
_________________________________________________________________
gru_1 (GRU)                  (None, 32)                12768     
_________________________________________________________________
dense_1 (Dense)              (None, 256)               8448      
_________________________________________________________________
dense_2 (Dense)              (None, 71)                18247     
Total params: 834,363
Trainable params: 39,463
Non-trainable params: 794,900
_________________________________________________________________


In [39]:
## Creating X and Y variables for training. X = Items or item 'feature'. Y = 'product' or product category

X = []
for i in range(len(tranzact_data_labelled_final)):
    
    X.append(tranzact_data_labelled_final.iloc[i]['item'])
#Q = np.array(tranzact_data_3["product"])
Y = tranzact_data_labelled_final['product']
print(X[1])

print(Y[1])


accumulator model a as 5126 3 4 connection emerson make
cooling


In [40]:
#train test split in 80% / 20% ratio

from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20, random_state = 5)


In [41]:
## X_train and X_test word embedding :

X_train_tokens = tokenizer_obj_1.texts_to_sequences(X_train)
X_test_tokens = tokenizer_obj_1.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_tokens, maxlen = max_length, padding = 'post')
X_test_pad = pad_sequences(X_test_tokens, maxlen = max_length, padding = 'post')

In [44]:
print(X_train_pad.shape)
print(X_test_pad.shape)
print(max_length)
print(vocab_size_1)

(9442, 121)
(2361, 121)
121
7949


In [42]:
from sklearn.preprocessing import LabelBinarizer

encoder = LabelBinarizer()

Y_train_coded = encoder.fit_transform(Y_train)
Y_test_coded = encoder.fit_transform(Y_test)

print(Y_train_coded.shape)
print(Y_test_coded.shape)


(9442, 71)
(2361, 69)


In [43]:
model_2.fit(X_train_pad,Y_train_coded,batch_size=128,epochs=25,
          validation_split=0.2)

Train on 7553 samples, validate on 1889 samples
Epoch 1/25
7553/7553 [==============================] - 8s 1ms/step - loss: 3.8448 - acc: 0.1392 - val_loss: 3.4788 - val_acc: 0.1122
Epoch 2/25
7553/7553 [==============================] - 8s 1ms/step - loss: 3.4929 - acc: 0.1354 - val_loss: 3.4630 - val_acc: 0.1509
Epoch 3/25
7553/7553 [==============================] - 7s 923us/step - loss: 3.4860 - acc: 0.1431 - val_loss: 3.4613 - val_acc: 0.1509
Epoch 4/25
7553/7553 [==============================] - 8s 1ms/step - loss: 3.4865 - acc: 0.1431 - val_loss: 3.4619 - val_acc: 0.1509
Epoch 5/25
7553/7553 [==============================] - 8s 1ms/step - loss: 3.4848 - acc: 0.1431 - val_loss: 3.4630 - val_acc: 0.1509
Epoch 6/25
7553/7553 [==============================] - 7s 946us/step - loss: 3.4842 - acc: 0.1431 - val_loss: 3.4652 - val_acc: 0.1509
Epoch 7/25
7553/7553 [==============================] - 7s 927us/step - loss: 3.4851 - acc: 0.1431 - val_loss: 3.4650 - val_acc: 0.1509
Epoch 8/